In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/kunmokim/Documents/classes/290C/git/chisel_sandbox/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

In [3]:
class RegNextModule extends Module {
  val io = IO(new Bundle {
    val in  = Input (UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  // register bitwidth is inferred from io.out
  io.out := RegNext(io.in + 1.U)
}

class RegNextModuleTester(c: RegNextModule) extends PeekPokeTester(c) {
  for (i <- 0 until 100) {
    poke(c.io.in, i)
    step(1)
    expect(c.io.out, i+1)
  }
}
assert(chisel3.iotesters.Driver(() => new RegNextModule) { c => new RegNextModuleTester(c) })
println("SUCCESS!!")

[info] [0.001] Elaborating design...
[info] [0.040] Done elaborating.
Total FIRRTL Compile Time: 145.0 ms
Total FIRRTL Compile Time: 7.4 ms
End of dependency graph
Circuit state created
[info] [0.001] SEED 1541022802677
test cmd2WrapperHelperRegNextModule Success: 100 tests passed in 105 cycles taking 0.036519 seconds
[info] [0.031] RAN 100 CYCLES PASSED
SUCCESS!!


defined class RegNextModule
defined class RegNextModuleTester